In [ ]:
%config InlineBackend.figure_format = 'retina'

# grid

## preds

In [ ]:
%run -n main
task_tests = {
    _: list(load_task(_, PUBLIC, TEST))
    for _ in TASKS
}

In [ ]:
%run -n main
for conf in GRID_CONFS:
    for task in TASKS:
        exp_dir = find_grid_exp_dir(EXPS_DIR, conf.id, conf.model, task)
        if not exp_dir:
            continue

        if not grid_exp_finised(exp_dir, task):
            continue
            
        if grid_preds_exist(conf.id, task):
            continue

        log('id: %r, task: %r', conf.id, task)
        preds = infer_jiant(exp_dir, task, task_tests[task], batch_size=16)
        dump_grid_preds(conf.id, task, preds)

## eval

In [ ]:
%run -n main
task_targets = {
    _: list(load_task(_, PRIVATE, TEST))
    for _ in TASKS
}

In [ ]:
%run -n main
conf_task_scores = []
for conf in log_progress(GRID_CONFS):
    for task in TASKS:
        if grid_preds_exist(conf.id, task):
            preds = list(load_grid_preds(conf.id, task))
            metrics = eval(task, preds, task_targets[task])
            score = select_score(task, metrics)
            conf_task_scores.append([conf.id, task, score])

## leaderboard

In [ ]:
%run -n main
records = parse_tsv(LEADERBOARD_LINES)
leaderboard = list(parse_leaderboard(records))

## show

In [ ]:
%run -n main
show_grid_scores(leaderboard, conf_task_scores)